In [ ]:
from ultralytics import YOLO
import torch.nn as nn
import torch

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# YOLO의 Detect 모듈 수정: 마지막 레이어를 6개의 파라미터 (cx, cy, w, h, angle)로 확장
class RotatedYOLO(nn.Module):
    def __init__(self, base_model):
        super(RotatedYOLO, self).__init__()
        self.base_model = base_model

        # YOLO의 Detect 레이어 가져오기
        self.detect_layer = self.base_model.model.model[-1]

        # YOLO의 기존 클래스 개수
        num_classes = self.detect_layer.nc

        # 새로운 Conv 레이어 추가: (cx, cy, w, h, angle) + obj_conf + class_probs
        in_channels = self.detect_layer.cv3[-1][2].out_channels  # 마지막 Conv 레이어의 출력 채널
        out_channels = 5 + 1 + num_classes  # (cx, cy, w, h, angle, obj_conf + class_probs)

        # 새로운 Conv2d 레이어 생성
        self.new_conv = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1))

        # Detect 레이어의 cv3 수정
        self.detect_layer.cv3[-1].add_module('new_conv', self.new_conv)

    def forward(self, x):
        # YOLO의 forward 과정을 수행
        return self.base_model(x)

    def train(self, *args, **kwargs):
        # YOLO의 기본 train 메소드 호출
        self.base_model.train(*args, **kwargs)

        # 필요한 커스텀 손실 함수를 train 루틴에 포함하기 위해 YOLO의 train 메소드 수정
        self.base_model.loss = self.custom_loss  # 손실 함수를 커스텀으로 변경

    # 커스텀 손실 함수 정의
    def custom_loss(self, preds, targets):
        # 바운딩 박스 손실 계산 (cx, cy, w, h)
        bbox_loss = nn.MSELoss()(preds[:, :4], targets[:, :4])  # (cx, cy, w, h)

        # 각도 손실 계산 (angle)
        angle_loss = nn.MSELoss()(preds[:, 4], targets[:, 4])  # angle

        # 총 손실 계산 (기존 바운딩 박스 손실 + 각도 손실)
        total_loss = bbox_loss + 0.1 * angle_loss  # 0.1은 각도 손실 가중치
        return total_loss


# 기존 모델에 수정된 구조 적용
rotated_model = RotatedYOLO(model)

# 학습 시작 (rotated_model을 사용하여 훈련)
results = rotated_model.train(
    data='data.yaml',  # 데이터 설정 파일
    epochs=50,         # 학습 epoch 수
    imgsz=256,         # 입력 이미지 크기
    batch=16,          # 배치 크기
    name='rotated_ship_detection',  # 실험 이름
    device=0           # GPU 사용 설정
)


In [25]:
import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# 모델의 전체 구조를 요약해서 출력
print(model)
# print(model.model)
# print(model.model.model)
# print(model.model.model[-1])



YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [8]:

# 모델의 레이어 이름과 모듈을 모두 출력
for name, module in model.model.named_modules():
    print(f"Layer Name: {name}, Module: {module}")


Layer Name: , Module: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): 

In [10]:
# YOLO의 Detect 모듈 출력
detect_layer = model.model.model[-1]  # Detect 레이어에 접근
print(detect_layer)


Detect(
  (cv2): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1-2): 2 x Sequential(
      (0): Conv(
        (conv): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [44]:
import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

for i in range(len(model.model.model)):
    print(f"Layer {i}: {model.model.model[i]}")


Layer 0: Conv(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)
Layer 1: Conv(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)
Layer 2: C3k2(
  (cv1): Conv(
    (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0): C3k(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), str

In [52]:
import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# Backbone과 Head 모듈 가져오기
backbone = model.model.model[:-1]  # Head를 제외한 Backbone
head = model.model.model[-1]        # Head

# 입력 텐서 생성 (예: 1x3x256x256)
input_tensor = torch.randn(1, 3, 256, 256)

# Backbone에 입력 텐서를 전달하여 출력 텐서 생성
backbone_output = backbone(input_tensor)

TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [40]:
import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# 임의의 입력 텐서 생성 (배치 크기 1, 채널 수 3, 높이 256, 너비 256)
input_tensor = torch.randn(1, 3, 256, 256)

# 출력의 shape을 저장할 리스트
output_shapes = []

# 각 레이어의 출력을 캡처할 hook 함수 정의
def get_output_shape(module, input, output):
    # 출력의 shape을 추가하는 재귀 함수
    def append_shapes(output_item):
        if isinstance(output_item, (tuple, list)):
            for item in output_item:
                append_shapes(item)  # 재귀적으로 호출
        else:
            output_shapes.append(output_item.shape)  # 텐서인 경우 shape 저장

    append_shapes(output)  # 처음 호출

# 모델의 모든 레이어에 hook 등록
for layer in model.model.modules():
    if hasattr(layer, 'register_forward_hook'):
        layer.register_forward_hook(get_output_shape)

# Pass the input tensor through the model
with torch.no_grad():  # 기울기 계산을 하지 않도록 설정
    model(input_tensor)  # 출력이 필요 없으므로 할당하지 않음

# Print the output shapes
for i, shape in enumerate(output_shapes):
    print(f"Layer {i + 1} output shape: {shape}")



WARNING  torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.156823635101318. Dividing input by 255.
0: 256x256 (no detections), 41.0ms
Speed: 0.0ms preprocess, 41.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)
Layer 1 output shape: torch.Size([1, 64, 320, 320])
Layer 2 output shape: torch.Size([1, 64, 320, 320])
Layer 3 output shape: torch.Size([1, 128, 160, 160])
Layer 4 output shape: torch.Size([1, 128, 160, 160])
Layer 5 output shape: torch.Size([1, 128, 160, 160])
Layer 6 output shape: torch.Size([1, 128, 160, 160])
Layer 7 output shape: torch.Size([1, 32, 160, 160])
Layer 8 output shape: torch.Size([1, 32, 160, 160])
Layer 9 output shape: torch.Size([1, 32, 160, 160])
Layer 10 output shape: torch.Size([1, 32, 160, 160])
Layer 11 output shape: torch.Size([1, 32, 160, 160])
Layer 12 output shape: torch.Size([1, 32, 160, 160])
Layer 13 output shape: torch.Size([1, 32, 160, 160])
Layer 14 output shape: torch.Size([1, 32, 160, 160])
Layer 15 outp